In [11]:
import pandas as pd

from Bio import pairwise2
from Bio.pairwise2 import format_alignment

In [9]:
file_path = '/home/xev/playground/covid-tutorial/data/spike_data.xlsx'

In [10]:
spike_data = pd.read_excel(file_path, engine='openpyxl')
spike_data

,pnum,pcode,ptype,pseq
0,1,7dx1_A,Spike glycoprotein,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
1,2,7dx1_C,Spike glycoprotein,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
2,3,7cak_A,Spike glycoprotein,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
3,9,7ddd_A,Spike glycoprotein,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
4,10,7ddd_C,Spike glycoprotein,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
...,...,...,...,...
199,1181,7bnn_C,Spike glycoprotein,MGILPSPGMPALLSLVSLLSVLLMGCVAETGMFVFLVLLPLVSSQC...
200,1185,7ek6_A,Spike protein S2,FNGIGVTQNVLYENQKLIANQFNSAIGKIQDSLSSTASALGKLQDV...
201,1186,7ek6_B,Spike protein S2,SVVNIQKEIDRLNEVAKNLNESLIDLQELGKYEQYIK
202,1194,7lqv_A,Spike glycoprotein,MGILPSPGMPALLSLVSLLSVLLMGCVAETGTQCVNLTTRTQLPPA...


In [15]:
count = 0

spike_network = pd.DataFrame(columns=['seq1', 'seq2', 'alignment_score', 'alignment'])

for i in range(0, len(spike_data)):
    for j in range(i+1, len(spike_data)):
        alignments = pairwise2.align.globalxx(spike_data['pseq'][i], spike_data['pseq'][j])
        count += 1
        spike_network.loc[count] = [spike_data['pcode'][i], spike_data['pcode'][j], alignments[0][2], alignments[0][4]]

In [16]:
spike_network

,seq1,seq2,alignment_score,alignment
1,7dx1_A,7dx1_C,1299.0,1300
2,7dx1_A,7cak_A,1217.0,1307
3,7dx1_A,7ddd_A,1244.0,1333
4,7dx1_A,7ddd_C,1243.0,1333
5,7dx1_A,6lzg_B,209.0,1302
...,...,...,...,...
20702,7ek6_A,7lqv_A,53.0,1324
20703,7ek6_A,7lqw_E,53.0,1323
20704,7ek6_B,7lqv_A,37.0,1324
20705,7ek6_B,7lqw_E,37.0,1323


In [22]:
spike_network['percent'] = spike_network['alignment_score'] / spike_network['alignment'] * 100
spike_network

,seq1,seq2,alignment_score,alignment,percent
1,7dx1_A,7dx1_C,1299.0,1300,99.923077
2,7dx1_A,7cak_A,1217.0,1307,93.114002
3,7dx1_A,7ddd_A,1244.0,1333,93.323331
4,7dx1_A,7ddd_C,1243.0,1333,93.248312
5,7dx1_A,6lzg_B,209.0,1302,16.052227
...,...,...,...,...,...
20702,7ek6_A,7lqv_A,53.0,1324,4.003021
20703,7ek6_A,7lqw_E,53.0,1323,4.006047
20704,7ek6_B,7lqv_A,37.0,1324,2.794562
20705,7ek6_B,7lqw_E,37.0,1323,2.796674


In [24]:
with pd.ExcelWriter('/home/xev/playground/covid-tutorial/data/spike_network.xlsx', engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    spike_network.to_excel(writer, sheet_name='spike_network', index=False)

In [26]:
spike_network_50 = spike_network[spike_network['percent'] >= 50]
spike_network_50

,seq1,seq2,alignment_score,alignment,percent
1,7dx1_A,7dx1_C,1299.0,1300,99.923077
2,7dx1_A,7cak_A,1217.0,1307,93.114002
3,7dx1_A,7ddd_A,1244.0,1333,93.323331
4,7dx1_A,7ddd_C,1243.0,1333,93.248312
6,7dx1_A,7k8u_A,1243.0,1332,93.318318
...,...,...,...,...,...
20695,7bnn_A,7lqv_A,1261.0,1367,92.245794
20696,7bnn_A,7lqw_E,1261.0,1366,92.313324
20699,7bnn_C,7lqv_A,1260.0,1367,92.172641
20700,7bnn_C,7lqw_E,1260.0,1366,92.240117


In [27]:
with pd.ExcelWriter('/home/xev/playground/covid-tutorial/data/spike_network_50.xlsx', engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    spike_network_50.to_excel(writer, sheet_name='spike_network_50', index=False)

In [33]:
spike_network_50[['seq1', 'seq2']].to_csv('/home/xev/playground/covid-tutorial/data/spike_network_50.tsv', sep='\t', index=False)

In [28]:
spike_network_80 = spike_network[spike_network['percent'] >= 80]
spike_network_80

,seq1,seq2,alignment_score,alignment,percent
1,7dx1_A,7dx1_C,1299.0,1300,99.923077
2,7dx1_A,7cak_A,1217.0,1307,93.114002
3,7dx1_A,7ddd_A,1244.0,1333,93.323331
4,7dx1_A,7ddd_C,1243.0,1333,93.248312
6,7dx1_A,7k8u_A,1243.0,1332,93.318318
...,...,...,...,...,...
20695,7bnn_A,7lqv_A,1261.0,1367,92.245794
20696,7bnn_A,7lqw_E,1261.0,1366,92.313324
20699,7bnn_C,7lqv_A,1260.0,1367,92.172641
20700,7bnn_C,7lqw_E,1260.0,1366,92.240117


In [30]:
with pd.ExcelWriter('/home/xev/playground/covid-tutorial/data/spike_network_80.xlsx', engine='openpyxl', mode='w') as writer:
    spike_network_80.to_excel(writer, sheet_name='spike_network_80', index=False)

In [34]:
spike_network_80[['seq1', 'seq2']].to_csv('/home/xev/playground/covid-tutorial/data/spike_network_80.tsv', sep='\t', index=False)